In [20]:
import pyodbc as pyodbc
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline
import yfinance as yf
from config import ROOT_DIR

#### Connect to sql server db and create stocks table

In [12]:
DRIVER = 'SQL Server'
SERVER_NAME = 'LAPTOP-S1EISQB0'
DB_NAME = 'newdb'

In [13]:
conn_str = (f"Driver={DRIVER};Server={SERVER_NAME};Database={DB_NAME};Trusted_Connection=yes;")
conn = pyodbc.connect(conn_str)
conn.autocommit = True 
cursor = conn.cursor()

In [41]:
query = """
use newdb;
drop table if exists stocks;
create table stocks (
Date date not null PRIMARY KEY,
Open_Price numeric(9, 6) not null,
High_Price numeric(9, 6) not null,
Low_Price numeric(9, 6) not null,
Close_Price numeric(9, 6) not null,
Adj_Close numeric(9, 6) not null,
Volume int not null
);
"""""

In [42]:
cursor.execute(query)

#### Get stocks data for Bank of America from yahoo finance api

In [21]:
yf.pdr_override()
startdate = datetime(2006,1,1)
enddate = datetime(2016,1,1)

In [22]:
data = pdr.get_data_yahoo(["BAC"], start=startdate, end=enddate)

[*********************100%***********************]  1 of 1 completed


In [24]:
data.reset_index(inplace=True)

In [25]:
data.to_csv(rf"{ROOT_DIR}\data\stocks.csv", index=False)

In [27]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-01-03,46.919998,47.180000,46.150002,47.080002,33.170311,16296700
1,2006-01-04,47.000000,47.240002,46.450001,46.580002,32.818035,17757900
2,2006-01-05,46.580002,46.830002,46.320000,46.639999,32.860306,14970700
3,2006-01-06,46.799999,46.910000,46.349998,46.570000,32.810974,12599800
4,2006-01-09,46.720001,46.970001,46.360001,46.599998,32.832123,15619400


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       2517 non-null   datetime64[ns]
 1   Open       2517 non-null   float64       
 2   High       2517 non-null   float64       
 3   Low        2517 non-null   float64       
 4   Close      2517 non-null   float64       
 5   Adj Close  2517 non-null   float64       
 6   Volume     2517 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 137.8 KB


#### Insert data to sql server table stocks

In [46]:
cols = ["Date", "Open_Price", "High_Price", "Low_Price", "Close_Price", "Adj_Close", "Volumne"]
records = data.values.tolist()

In [47]:
sql_insert = """ 
    insert into stocks
    values (?, ?, ?, ?, ?, ?, ?)
"""

In [48]:
cursor.executemany(sql_insert, records)